In [39]:
import pandas as pd
from operator import itemgetter
import ast
import numpy as np


In [40]:
seen_df = pd.read_csv('data/0a_labelled_documents.csv')
unseen_df = pd.read_csv('data/0b_unlabelled_documents.csv')

df = (pd.concat([seen_df,unseen_df])
      .sort_values('id')
      .sample(frac=1, random_state=1)
      .reset_index(drop=True)
     )

seen_index = df[df['seen']==1].index
unseen_index = df[df['seen']==0].index
print(df.shape)

# Get a list of predictions for the unseen documents
y_pred = np.load('data/predictions.npy')
df['prediction'] = df['relevant']
df.loc[unseen_index,"prediction"] = y_pred[:,1]

df = df[df['prediction']>0.5]
print(df.shape)

df.head()

(699525, 14)
(21965, 15)


,id,content,title,PY,wosarticle__de,wosarticle__wc,relevant,seen,1 - Adaptation,1 - Impact,1 - Mitigation,wosarticle__dt,tslug,UT__UT,prediction
52,4963912,Objective To explore the association between n...,Exploration of the association between meteoro...,2021.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,Article,explorationoftheassociationbetweenmeteorologic...,2-s2.0-85123003389,0.792216
69,4729030,Background. Acute gastroenteritis is a cause o...,ACUTE GASTROENTERITIS IN 3 COMMUNITY-BASED NUR...,1995.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,Article,acutegastroenteritisin3communitybasednursinghomes,WOS:A1995RY60500014,0.715006
80,4762134,Background: Despite declining age-adjusted str...,Mortality and Predictors of Death 1 Month and ...,2010.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,Article,mortalityandpredictorsofdeath1monthand3yearsaf...,WOS:000273552100004,0.553015
99,678134,Weather factors are widely studied for their e...,Statistical Modeling Reveals the Effect of Abs...,2014.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,Article,statisticalmodelingrevealstheeffectofabsoluteh...,WOS:000337735100017,0.958598
113,304387,The aim of this study was to determine the hea...,A systematic review of the health effects of p...,2015.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,Review,asystematicreviewofthehealtheffectsofprenatale...,WOS:000364521200008,0.588241


In [42]:
places = pd.read_csv('data/places.csv')
print(len(places.doc_id.unique()))

places = places[places['country_conf']>0.4]

print(len(places.doc_id.unique()))

gm_docs = df.loc[
    (df['title'].str.lower().str.contains("gulf of mexico")) |
    (df['content'].str.lower().str.contains("gulf of mexico")),
    "id"
]
geocolumns = ["word", "country_conf", "feature_code","lat","lon","place_name","feature_class","geonameid"]
gm = pd.DataFrame({"doc_id": gm_docs})
gm[geocolumns] = ["Gulf of Mexico",0.8,"GULF", 25, -90, "Gulf of Mexico", "H", 3523271]


lab_docs = df.loc[
    (df['title'].str.lower().str.contains("labrador sea")) |
    (df['content'].str.lower().str.contains("labrador sea")),
    "id"
]
geocolumns = ["word", "country_conf", "feature_code","lat","lon","place_name","feature_class","geonameid"]
lab = pd.DataFrame({"doc_id": lab_docs})
lab[geocolumns] = ["Labrador Sea",0.8,"SEA", 57, -55, "Labrador Sea", "H", 3424929]

baf_docs = df.loc[
    (df['title'].str.lower().str.contains("baffin bay")) |
    (df['content'].str.lower().str.contains("baffin bay")),
    "id"
]
geocolumns = ["word", "country_conf", "feature_code","lat","lon","place_name","feature_class","geonameid"]
baf = pd.DataFrame({"doc_id": baf_docs})
baf[geocolumns] = ["Baffin Bay",0.8,"BAY", 74, -68, "Baffin Bay", "H", 3831554]

ok_docs = df.loc[
    (df['title'].str.lower().str.contains("sea of okhotsk")) |
    (df['content'].str.lower().str.contains("sea of okhotsk")),
    "id"
]
geocolumns = ["word", "country_conf", "feature_code","lat","lon","place_name","feature_class","geonameid"]
ok = pd.DataFrame({"doc_id": baf_docs})
ok[geocolumns] = ["Sea of Okhotsk",0.8, "SEA", 55, 150, "Sea of Okhotsk", "H", 2127380]

places = pd.concat([places, gm, lab, baf, ok])

places.sort_values('country_conf').head()

28196
26366


,word,spans,country_predicted,country_conf,admin1,lat,lon,country_code3,geonameid,place_name,feature_class,feature_code,doc_id
98814,RN,"[{'start': 524, 'end': 526}]",BRA,0.40238,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4090647
118584,Bihar,"[{'start': 86, 'end': 91}]",IND,0.40238,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1629390
118582,Bihar,"[{'start': 49, 'end': 54}]",IND,0.40238,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1629390
104189,Hamilton,"[{'start': 1000, 'end': 1008}]",CAN,0.40238,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5154081
102616,San Martin,"[{'start': 554, 'end': 564}]",PER,0.40238,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3757869


In [43]:
geocolumns = ["feature_code","lat","lon","place_name","feature_class","geonameid"]
places.loc[places["word"]=="Pakistan", geocolumns]=["PCLI",30,70,"Islamic Republic of Pakistan","A",1168579]
places.loc[places["word"]=="Colombia", geocolumns]=["PCLI",4,-73.25,"Colombia","A",3686110]
places.loc[places["word"]=="Argentina", geocolumns]=["PCLI",-34,-64,"Argentine Republic","A",3865483]
places.loc[places["word"]=="Sahara", geocolumns] = ["DSRT", 26, 13, "Sahara", "T", 2212709]
places.loc[places["word"]=="Alps",geocolumns] = ["MTS", 46.41667, 10, "Alps", "T", 2661786]
places.loc[places["word"]=="Mediterranean Sea",geocolumns] = ["SEA", 35, 20, "Mediterranean Sea", "T", 2661786]
places.loc[places["word"]=="MEDITERRANEAN",geocolumns] = ["SEA", 35, 20, "Mediterranean Sea", "T", 2661786]
places.loc[places["word"]=="East China",geocolumns] = ["PCLI", 35, 105, "China", "A", 1814991]
places.loc[places["word"]=="South China",geocolumns] = ["PCLI", 35, 105, "China", "A", 1814991]
places.loc[places["word"]=="Great Lakes",geocolumns] = ["LK", 45.68751, -84.43753, "Great Lakes", "H", 4994594]
places.loc[places["word"]=="Catalonia",geocolumns] = ["ADM1", 41.82046, 1.86768, "Catalunya", "A", 3336901]
places.loc[places["word"]=="South Pacific",geocolumns] = ["OCN", -45, -130, "South Pacific Ocean", "H", 4030483]
places.loc[places["word"]=="Gulf Coast",geocolumns] = ["AREA", 29.36901, -95.00565, "Gulf Coast", "L", 7287689]
places.loc[places["word"]=="Gulf coast",geocolumns] = ["AREA", 29.36901, -95.00565, "Gulf Coast", "L", 7287689]
places.loc[places["word"]=="Hainan Island",geocolumns] = ["ISL", 19.2, 109.7, "Hainan Dao", "T", 1809055]
places.loc[places["word"]=="Red Sea",geocolumns] = ["SEA", 20.26735, 38.53455, "Red Sea", "H", 350155]
places.loc[places["word"]=="Himalayan",geocolumns] = ["MTS", 28,84, "Himalayas", "T", 1252558]
places.loc[places["word"]=="Himalayas",geocolumns] = ["MTS", 28,84, "Himalayas", "T", 1252558]
places.loc[places["word"]=="North America's",geocolumns] = ["CONT", 46.07323, -100.54688, "North America", "L", 6255149]
places.loc[places["word"]=="Atlantic Ocean",geocolumns] = ["OCN", 10, -25, "Atlantic Ocean", "H", 3373405]
places.loc[places["word"]=="Scandinavia",geocolumns] = ["RGN", 63, 12, "Scandinavia", "L", 2614165]
places.loc[places["word"]=="California (USA",geocolumns] = ["ADM1", 37.25022, -119.75126, "California", "A", 5332921]
places.loc[places["word"]=="California, USA",geocolumns] = ["ADM1", 37.25022, -119.75126, "California", "A", 5332921]
places.loc[places["word"]=="North Pacific",geocolumns] = ["OCN", 30, -170, "North Pacific Ocean", "H", 4030875]
places.loc[places["word"]=="Huai",geocolumns] = ["STM", 33.133333, 118.5, "Huai He", "H", 1807690]
places.loc[places["word"]=="Washington, DC",geocolumns] = ["PPLC", 38.89511, -77.03637, "Washington", "P", 4140963]
places.loc[places["word"]=="Messinian",geocolumns] = ["ADM2", 37.25, -21.83333, "Nomos Messinias", "A", 257149]
places.loc[places["word"]=="Ionian Sea",geocolumns] = ["SEA", 39, 19, "Ionian Sea", "H", 2463713]
places.loc[places["word"]=="NYC",geocolumns] = ["PPL", 40.71427, -74.00597, "New York City", "P", 5128581]
places.loc[places["word"]=="Indian Ocean",geocolumns] = ["OCN", -10, 70, "Indian Ocean", "P", 1545739]
places.loc[places["word"]=="North Sea",geocolumns] = ["SEA", 55, 3, "North Sea", "P", 2960848]
places.loc[places["word"]=="Philippine Sea",geocolumns] = ["SEA", 20, 135, "Philippine Sea", "P", 1818190]
places.loc[places["word"]=="Black Sea",geocolumns] = ["SEA", 43, 34, "Black Sea", "H", 630673]
places.loc[places["word"]=="Coral Sea",geocolumns] = ["SEA", -20, 155, "Coral Sea", "H", 2194166]
places.loc[places["word"]=="Timor Sea",geocolumns] = ["SEA", -11, 127, "Timor Sea", "H", 2078065]
places.loc[places["word"]=="Hudson Bay",geocolumns] = ["BAY", 60, -85, "Hudson Bay", "H", 5978134]
places.loc[places["word"]=="Bering Sea",geocolumns] = ["SEA", 60, -175, "Bering Sea", "H", 4031788]
places.loc[places["word"]=="Okhotsk Sea",geocolumns] = ["SEA", 55, 150, "Sea of Okhotsk", "H", 2127380]

places.loc[places["place_name"]=="Central Upper Nile",geocolumns] = ["ADM1", 10, 32.7, "Upper Nile", "A", 381229]


places.loc[places["place_name"]=="Gobolka Woqooyi Galbeed","place_name"] = "Woqooyi Galbeed"

places = places[places["place_name"]!="Pacific County"]
places = places.loc[places["word"]!="B.V."]
places = places[places["word"]!="MMT"]
places = places[places["word"]!="Yellow"]
places = places[places["word"]!="Hadley"]
places = places[places["word"]!="Western North"]
places = places[places["word"]!="colonies"]
places = places[places["word"]!="TN"]
places = places[places["word"]!="NH"]
places = places[places["word"]!="Mn"]
places = places[places["word"]!="Tx"]
places = places[places["word"]!="TX"]
places = places[places["word"]!="Tn"]
places = places[places["word"]!="FL"]
places = places[places["word"]!="Spartina"]
places = places[places["word"]!="Tamarix"]
places = places[places["word"]!="Eurasia"]
places = places[places["word"]!="Phillyrea"]
places = places[places["word"]!="N-15"]
places = places[places["word"]!="LT50"]
places = places[places["word"]!="POSEIDON"]
places = places[places["word"]!="LC50"]
places = places[places["word"]!="El Nio"]
places = places[places["word"]!="La Nia"]
places = places[places["word"]!="Red"]
places = places[places["word"]!="Gulf Stream"]
places = places[places["word"].str.len()>2]
places = places[places["word"]!="NH 1"]
places = places[places["word"]!="Quercus"]
places = places[(places["word"]!="ZJP")]


places.lat = places.lat.astype(float)
places.lon = places.lon.astype(float)

places = places.drop_duplicates(["doc_id","word"])

print(places.shape)

places.to_csv("data/clean_place_df.csv",index=False)


(55951, 13)


In [44]:
import pycountry_convert as pc
def get_cont(x):
    continents = {
        'NA': 'North America',
        'SA': 'South America', 
        'AS': 'Asia',
        'OC': 'Oceania',
        'AF': 'Africa',
        'EU': 'Europe'
    }
    try:
        return continents[pc.country_alpha2_to_continent_code(pc.country_alpha3_to_country_alpha2(x['country_predicted']))]
    except:
        return None
places['continent'] = places.apply(lambda x: get_cont(x), axis=1)
other_country_continents = [
    ("ESH","Africa"),
    ("TLS","Oceania"),
    ("XKX","Europe")
    
]
for country,continent in other_country_continents:
    places.loc[places['country_predicted']==country,"continent"] = continent
    
print(places.shape)

places.to_csv("data/clean_place_df.csv",index=False)
    

places.head()

(55951, 14)


,word,spans,country_predicted,country_conf,admin1,lat,lon,country_code3,geonameid,place_name,feature_class,feature_code,doc_id,continent
0,Shanghai,"[{'start': 162, 'end': 170}]",CHN,0.963741,Shanghai,31.22222,121.45806,CHN,1796236.0,Shanghai,P,PPLA,4963912,Asia
8,Thailand,"[{'start': 57, 'end': 65}]",THA,0.948191,NaN,15.50000,101.00000,THA,1605651.0,Kingdom of Thailand,A,PCLI,4695222,Asia
9,Bangkok,"[{'start': 302, 'end': 309}]",THA,0.961249,Bangkok,13.75398,100.50144,THA,1609350.0,Bangkok,P,PPLC,4695222,Asia
11,Israel,"[{'start': 285, 'end': 291}]",ISR,0.948191,NaN,31.50000,34.75000,ISR,294640.0,State of Israel,A,PCLI,4762134,Asia
13,Basel,"[{'start': 2553, 'end': 2558}]",CHE,0.938224,Basel-City,47.55773,7.59361,CHE,7285161.0,Basel,A,ADM3,4762134,Europe


In [45]:
pubs = [
    ("S. Karger AG, Basel",["Basel","Switzerland"]),
    ("Licensee MDPI, Basel",["Basel","Switzerland"])
]
for p, place in pubs:
    pub_docs = df.loc[df.content.str.contains(p)]
    remove = places[(places.doc_id.isin(pub_docs['id'])) & (places['place_name'].isin(place))]
    print(remove.shape[0])
    places = places.drop(index=remove.index)

    
print(places.shape)
places.to_csv("data/clean_place_df.csv",index=False)
places.head()

13
190
(55748, 14)


,word,spans,country_predicted,country_conf,admin1,lat,lon,country_code3,geonameid,place_name,feature_class,feature_code,doc_id,continent
0,Shanghai,"[{'start': 162, 'end': 170}]",CHN,0.963741,Shanghai,31.22222,121.45806,CHN,1796236.0,Shanghai,P,PPLA,4963912,Asia
8,Thailand,"[{'start': 57, 'end': 65}]",THA,0.948191,NaN,15.50000,101.00000,THA,1605651.0,Kingdom of Thailand,A,PCLI,4695222,Asia
9,Bangkok,"[{'start': 302, 'end': 309}]",THA,0.961249,Bangkok,13.75398,100.50144,THA,1609350.0,Bangkok,P,PPLC,4695222,Asia
11,Israel,"[{'start': 285, 'end': 291}]",ISR,0.948191,NaN,31.50000,34.75000,ISR,294640.0,State of Israel,A,PCLI,4762134,Asia
14,Singapore,"[{'start': 74, 'end': 83}]",SGP,0.946707,NaN,1.36667,103.80000,SGP,1880251.0,Republic of Singapore,A,PCLI,678134,Asia


In [46]:
places[places['doc_id']==612295]

,word,spans,country_predicted,country_conf,admin1,lat,lon,country_code3,geonameid,place_name,feature_class,feature_code,doc_id,continent
13147,Zhejiang,"[{'start': 52, 'end': 60}]",CHN,0.951695,Zhejiang,29.16667,120.00000,CHN,1784764.0,Zhejiang Sheng,A,ADM1,612295,Asia
13148,East China,"[{'start': 71, 'end': 81}]",CHN,0.687582,Shanghai,35.00000,105.00000,CHN,1814991.0,China,A,PCLI,612295,Asia
13149,China,"[{'start': 127, 'end': 132}]",CHN,0.687582,Hubei,30.52047,114.39637,CHN,6620465.0,China University of Geosciences,S,SCHC,612295,Asia
